# SAT Bootcamp Analysis 2
### Goal:
Evenly divide new Ckodon students into random groups of 30.
Assign group labels from I to Z.

### Steps
1. Import Google Form data into Pandas DataFrame.
2. Clean the data:
    1. Clean leading and trailing white space from all string columns.
    2. Search for and print out duplicate rows.
    3. Search for and print out invalid email addresses.
3. Group the students:
    1. Define function for randomly assigning groups.
    2. Create \`SAT Group\` column and assign groups in column.
    3. Sort students by group.
4. Export resulting DataFrame to Excel Workbook.
5. Send assigned Group to each student via Email

## Import Google Form data

In [ ]:
# import modules for data analysis
import pandas as pd
from re import search
from random import randint, seed

# import student data

# students who have taken the SAT before
students_bf = pd.read_excel("../data/batch-2/Ckodon Activity Review Form - Before (BATCH 2).xlsx")

# students who have not been assigned an SAT group
students_ng = pd.read_excel("../data/batch-2/Ckodon Activity Review Form - No Group (BATCH 2).xlsx")


# test import success
print(students_bf.head())
print(students_ng.head())

## Clean the data

### Remove leading and trailing white space from all string columns.
Remove leading and trailing whitespace from the `Full Name` and `Email` column.


In [163]:
students_bf["Full Name"] = students_bf["Full Name"].apply(str.strip)
students_bf["Email"] = students_bf["Email"].apply(str.strip)
students_ng["Full Name"] = students_ng["Full Name"].apply(str.strip)
students_ng["Email"] = students_ng["Email"].apply(str.strip)

### Search and print out duplicate rows
Identify all rows with the same Full Name and Email or Full Name and WhatsApp Number.
Duplicate rows will be manually inspected in MS Excel and removed from the data set after identification.


In [164]:
# define function for finding duplicates
def print_duplicates(dataframe):
    """Prints all duplicate rows in `dataframe`.

    Parameters
    ==========
    dataframe : pandas.core.frame.DataFrame
        A pandas DataFrame containing each student's information.

    Returns
    =======
    None
    """
    # create copy of `dataframe`
    dataframe_cp = dataframe.copy()

    # extract relevant columns as lists for easy searching
    dataframe_cp["Full Name"] = dataframe_cp["Full Name"].apply(str.lower)  # remove casing
    dataframe_cp["Email"] = dataframe_cp["Full Name"].apply(str.lower)  # remove casing
    names = [name.lower() for name in dataframe_cp["Full Name"]]
    emails = [email.lower() for email in dataframe_cp["Email"]]
    whatsapp_nos = list(dataframe_cp["WhatsApp Number"])

    for row in dataframe_cp.index:
        name = dataframe_cp.loc[row]["Full Name"]
        email = dataframe_cp.loc[row]["Email"]
        whatsapp_no = dataframe_cp.loc[row]["WhatsApp Number"]
        row_excel = row + 2  # row number in MS Excel

        if names.count(name) > 1:
            if emails.count(email) > 1 or whatsapp_nos.count(whatsapp_no) > 1:
                no_of_duplicates = names.count(name)
                print(
                    f"DUPLICATE FOUND ({no_of_duplicates}): {row_excel} {dataframe['Full Name'][row]} {whatsapp_no} {dataframe['Email'][row]}")

In [ ]:
# search duplicate values from `students_bf`
print("----------Searching-STUDENTS_BF--------------")
print_duplicates(students_bf)

# search duplicate values from `students_ng`
print("\n\n----------Searching-STUDENTS_NG--------------")
print_duplicates(students_ng)

In [ ]:
# merge and search duplicates across sheets (after removing duplicates manually)
students = pd.concat([students_bf, students_ng], ignore_index=True)

print("----------Searching-STUDENTS_BF-and-STUDENTS_NG--------------")
print_duplicates(students)

### Search and print out invalid email addresses
Crucial as each student's group will be sent to them via email.

In [167]:
# define function for validating email addresses
def validate_address(email):
    """Return True if `email` is valid or False otherwise.

    Parameters
    ==========
    email: str
        The email address to be validated.

    Returns
    =======
    bool
        True means email address is valid. False means email address is invalid.
    """

    pattern = "[a-zA-Z0-9\-.+_]+@[a-zA-Z0-9\-.+_]+\.[a-zA-Z]{2,}"
    match = search(pattern, email)
    return bool(match)

In [ ]:
# call function on all addresses in email column

# students_bf
print("-------Invalid addresses in `students_bf`--------")
for row in students_bf.index:
    email = students_bf.loc[row]["Email"]
    if not validate_address(email):
        print(students_bf.loc[row])

# students_ng
print("-------Invalid addresses in `students_ng`--------")
for row in students_ng.index:
    email = students_ng.loc[row]["Email"]
    if not validate_address(email):
        print(students_ng.loc[row])

## Group the students

### Define function for randomly assigning groups

In [169]:
# recursive random number generator without repetition
previous = []  # list of previously generated values
seed(49)  # seed for random generator

def generate(a, b):
    """Returns pseudorandom integer between a and b if not already contained in a global list with name `previous`.

    Parameters
    ==========
    a : int
        Lower bound of the range within which generated number may lie (inclusive).
    b : int
        Upper bound of the range within which generated number may lie (inclusive).

    Returns
    =======
    int
        A pseudorandom number between a and b inclusive whose value is not already contained in `previous` list.
    """
    number = randint(a, b)
    if number not in previous:
        previous.append(number)
        return number
    else:
        return generate(a, b)

### Create \`SAT Group\` column and assign groups in column

In [170]:
# Create SAT Group column
students_bf["SAT Group"] = None
students_ng["SAT Group"] = None

In [ ]:
# Assign groups for `students_bf`
no_of_groups_a = round(len(students_bf) / 30)  # no of groups for students_bf
groups = [chr(i + 84) for i in range(no_of_groups_a)]

# for the no. of students there are, pick a random student and assign them a group.
for i in range(len(students_bf)):
    student = generate(0, len(students_bf) - 1)
    group = groups[i % len(groups)]
    students_bf["SAT Group"][student] = group

print("---------No-of-Students-per-Group--------")
for group in groups:
    print(group, list(students_bf["SAT Group"]).count(group))

In [ ]:
# Assign groups for `students_ng`
previous = []  # reset `previous` list for random number generator
students_ng["SAT Group"] = None  # create new column
no_of_groups_b = round(len(students_ng) / 30)  # no of groups for students_bf
groups = [chr(i + 84 + no_of_groups_a) for i in range(no_of_groups_b)]

# for the no. of students there are, pick a random student and assign them a group.
for i in range(len(students_ng)):
    student = generate(0, len(students_ng) - 1)
    group = groups[i % len(groups)]
    students_ng["SAT Group"][student] = group

print("---------No-of-Students-per-Group--------")
for group in groups:
    print(group, list(students_ng["SAT Group"]).count(group))

### Sort students by group

In [ ]:
students_bf.sort_values("SAT Group", inplace=True)
students_ng.sort_values("SAT Group", inplace=True)

print(students_bf)
print(students_ng)

## Export to Excel Workbook

In [174]:
students_bf.to_excel("../data/batch-2/output/Ckodon Activity Review Groups(BEFORE) [BATCH 2].xlsx", index=False)
students_ng.to_excel("../data/batch-2/output/Ckodon Activity Review Groups(NO GROUP) [BATCH 2].xlsx", index=False)

# Email Group Assignments to Students
Email the assigned group and corresponding WhatsApp group link to each student.

In [ ]:
# concatenate `students_bf` and `students_ng`
students = pd.concat([students_bf, students_ng], ignore_index=True)
students

In [182]:
# import modules for email
from smtplib import SMTP
from email.message import EmailMessage
from time import sleep
import json

# import server credentials and WhatsApp links from json file
credentials = json.loads(open("../data/credentials.json").read()) 

whatsapp_links = credentials["whatsapp_links"]

# instantiate SMTP client
server = credentials["server"]
port = credentials["port"]
username = credentials["username"]
password = credentials["password"]

smtp = SMTP(server, port)
smtp.ehlo()
smtp.starttls()
smtp.login(username, password)


# to prevent server overload (Gmail SMTP Error 421)
emails_sent = 0  # no. of emails sent so far
delay_s = 60  # delay in seconds


# for every student, create and send an email message
for row in students.index:
    student = students.loc[row]
    msg = EmailMessage()
    msg["From"] = "ckodontech@gmail.com"
    msg["To"] = student["Email"]
    msg["Subject"] = "Your Ckodon Activity Review Group"
    body = f"""Dear {student["Full Name"].title()},

You have been assigned to Ckodon Activity Review Group {student["SAT Group"]}.

You may use the link below to join your assigned group on WhatsApp:
{whatsapp_links[student["SAT Group"]]}

Join the group as soon as possible so that we can begin with the review. This link is meant to be used by you alone. Do not share with any other person.

Please do not reply to this email.

Best,
The Ckodon Foundation Team."""
    msg.set_content(body)

    # send email message
    try:
        smtp.send_message(msg)
        print("SENT")
        emails_sent += 1

        # wait 1 minute every 60 emails to prevent Gmail SMTP Error 421
        if emails_sent == 60:
            emails_sent = 0
            sleep(delay_s)
    except Exception as exception:
        print("Exception:", exception)
        print("Sending terminated at row number:", row)


SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
